In [6]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
import os

# Create a SparkConf and SparkContext
conf = SparkConf().setAppName("RemoveDuplicates")
sc = SparkContext(conf=conf)

# Read the text file into an RDD
file_path = "/home/basu/data/source.txt"
text_rdd = sc.textFile(file_path)

# Split each line into words and flatten the RDD
words_rdd = text_rdd.flatMap(lambda line: line.split(" "))

# Remove duplicates using distinct()
unique_words_rdd = words_rdd.distinct()
# Group words to find duplicates
grouped_rdd = words_rdd.map(lambda word: (word, 1)).groupByKey()
#grouped_rdd.collect()
# Filter out groups with a size greater than 1 (duplicates)
duplicates_rdd = grouped_rdd.filter(lambda x: len(x[1]) > 1).keys()


output_path = "/home/basu/data/removed_duplicates.txt"
output_path_duplicates = "/home/basu/data/duplicates.txt"


# Remove existing output directories
if os.path.exists(output_path):
    os.system("rm -r {}".format(output_path))
if os.path.exists(output_path_duplicates):
    os.system("rm -r {}".format(output_path_duplicates))


# Save the unique words as a text file

unique_words_rdd.coalesce(1).saveAsTextFile(output_path)
output_path_duplicates = "/home/basu/data/duplicates.txt"
duplicates_rdd.coalesce(1).saveAsTextFile(output_path_duplicates)



ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=RemoveDuplicates, master=local[*]) created by __init__ at /tmp/ipykernel_1240/341487731.py:8 

In [4]:
grouped_rdd.collect()

AttributeError: 'NoneType' object has no attribute 'setCallSite'

In [4]:
spark = SparkSession.builder.appName('x').getOrCreate()
data = [(1, 'John', 5000),
        (1, 'Alice', 6000),
        (1, 'Bob', 7000),
        (2, 'Charlie', 5500),
        (2, 'David', 6500),
        (3, 'Eva', 7500),
        (3, 'Frank', 8000),
        (3, 'Grace', 9000)]

columns = ['department', 'employee_name', 'salary']

df = spark.createDataFrame(data=data, schema=columns)
df.show()

+----------+-------------+------+
|department|employee_name|salary|
+----------+-------------+------+
|         1|         John|  5000|
|         1|        Alice|  6000|
|         1|          Bob|  7000|
|         2|      Charlie|  5500|
|         2|        David|  6500|
|         3|          Eva|  7500|
|         3|        Frank|  8000|
|         3|        Grace|  9000|
+----------+-------------+------+



In [7]:
window = Window.partitionBy('department').orderBy('salary')
df = df.withColumn('rank_number', row_number().over(window))
df.show()

NameError: name 'row_number' is not defined